# Example 2: Discrete + CVaR optimization with gaussian modelling - Unsuccessful solution

In this second example a very similar problem to the first one is solved, but not enough time is given to the solver so no integer solution is found.

## Data retrieval

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

### Get price data

In [3]:
df_prices = pd.read_csv('./data/weekly_example_1.csv', index_col=0, parse_dates=True)
df_prices.head()

,^GSPC,^DJI,^IXIC,^RUT,^FTSE,^N225,^HSI,^XAU,^GDAXI
Date,,,,,,,,,
1992-01-03,419.339996,3201.500000,592.650024,192.089996,2504.100098,22983.769531,4307.100098,79.180000,1604.050049
1992-01-10,415.100006,3199.500000,615.700012,199.309998,2477.899902,22381.900391,4348.899902,79.059998,1613.819946
1992-01-17,418.859985,3265.000000,626.849976,205.059998,2536.699951,21321.369141,4454.899902,82.949997,1669.290039
1992-01-24,415.480011,3232.800049,624.679993,205.419998,2510.399902,21072.150391,4600.100098,81.500000,1669.520020
1992-01-31,408.779999,3223.399902,620.210022,205.160004,2571.199951,22023.050781,4601.799805,83.529999,1685.699951


### Get Cash Flow data

For this example because of privacy concerns synthetic data will be artificially generated

In [4]:
from development.synth_data import generate_synth_income, generate_synth_expenses

In [5]:
arr_income = generate_synth_income(mean=2000, std=50)
arr_expenses = generate_synth_expenses(mean=1600, std=80)

## Returns Model

Now the model for the asset returns is chosen and fitted to the data. Note that in this case, because of the optimiztion model used, the model outputs prices directly instead of returns. 

In [6]:
from src.returns import LogNormalReturns

In [7]:
returns_model = LogNormalReturns()
returns_model.fit(df_prices=df_prices)

## CashFlow Model

Now the model for the income and the expenses is chosen and fitted to the data

In [8]:
from src.cashflows import NormalCashFlows

In [9]:
income_model = NormalCashFlows()
income_model.fit(cash_flow=arr_income)
expenses_model = NormalCashFlows()
expenses_model.fit(cash_flow=arr_expenses)

## Asset Allocation

For the last step, the optimization model is chosen and solved

In [11]:
from src.optimization.om_discrete_cvar_negativecash import create_model

These are the basic parameters for the model. They are included in the `constants.py` file, but will explicitally be included for explanatory purposes

In [37]:
N_SCENARIOS = 150 ## The more scenarios the more accurate the solution but more difficult to solve
HORIZON_PERIODS = 52 ## In this case, number of weeks to plan for. Note that only the first step is executed, but plannig several steps ahead provides more accurate and realistic deicisions
STARTING_CASH = 100_000
TRADING_FEE = 0.001
CVAR_ALPHA = 0.1 # The quantile for which the CVaR is calculated. The smaller the more the extreme cases will be taken into account. 
CVAR_GAMMA = 0.5 # Weight given to CVaR in the objective function 
# MINIUMUM_CASH = 10_000
# PROP_NEG_CASH = 0.05

In [38]:
df_prices.index[-1] # We make sure that prices data are up to date, in order to simulate prices starting at an up to date price

Timestamp('2024-01-05 00:00:00')

In [39]:
prices_syms = returns_model.predict(horizon=HORIZON_PERIODS, n_paths=N_SCENARIOS) # Prices are simulated
income_syms = income_model.predict(horizon=HORIZON_PERIODS, n_paths=N_SCENARIOS) # Income is simulated
expenses_syms = expenses_model.predict(horizon=HORIZON_PERIODS, n_paths=N_SCENARIOS) # Expenses are simulated

In [40]:


non_cash_assets = list(df_prices.columns)
sScenarios = list(range(N_SCENARIOS))
sInitialTime = [0]
sIntermediateTime = list(range(1, HORIZON_PERIODS))
sFinalTime = [HORIZON_PERIODS]
sNonInitialTimes = sIntermediateTime + sFinalTime
sNonFinalTimes = sInitialTime + sIntermediateTime
sTime = sInitialTime + sIntermediateTime + sFinalTime

data = {
    None: {
        'sInitialTime': {None: sInitialTime},
        'sIntermediateTime': {None: sIntermediateTime},
        'sFinalTime': {None: sFinalTime},
        'sNonCashAssets': {None: non_cash_assets},
        'sScenarios': {None: sScenarios},
        'pPrices': {(s_i,non_cash_assets[a_i], t_i): prices_syms[s_i,a_i,t_i] for s_i in sScenarios for t_i in sTime for a_i in range(len(non_cash_assets))},
        'pInitialNonCashAllocations': {a: 0 for a in non_cash_assets},
        'pInitialCashAllocations': {None: STARTING_CASH},
        'pIncome': {(s_i,t_i): income_syms[s_i, t_i] for s_i in sScenarios for t_i in sNonFinalTimes},
        'pExpense': {(s_i,t_i): expenses_syms[s_i, t_i] for s_i in sScenarios for t_i in sNonFinalTimes},
        'pTradeFee': {None: TRADING_FEE},
        'pCVaRAlpha': {None: CVAR_ALPHA},
        'pCVaRGamma': {None: CVAR_GAMMA},
        # 'pMinimumCash': {None: MINIUMUM_CASH},
        # 'pPropNegativeCash': {None: PROP_NEG_CASH},

    }
}


Now the optimization model is created, the data is fed, the solver is chosen and the model is solved

In [41]:
from src.optimization.om_discrete_cvar import create_model
from pyomo.environ import SolverFactory
from pyomo.opt.results import SolverStatus
import time

In [42]:

optimization_model = create_model()
instance = optimization_model.create_instance(data)
solver = SolverFactory('cbc') # The solver chosen here is open source and not very fast. Any solver that works with PyOMO can be chosen.
# Note that for the optimization to work the solver will need to be installed on the machine. 

solver.options['seconds'] = 300 # Solver will exit at 120s even if optimal solution is not found
                                # Note that each solver has different options with different names

print(f'Starting to solve...')
t1 = time.time()
result_obj = solver.solve(instance, tee=True)
result_obj.Solver.Status = SolverStatus.warning
instance.solutions.load_from(result_obj)   
t2 = time.time()
print(f"Solved in {t2-t1:.2f} s")

Starting to solve...
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -seconds 300 -printingOptions all -import /tmp/tmpr7i16kxl.pyomo.lp -stat=1 -solve -solu /tmp/tmpr7i16kxl.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 300
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve is modifying 9 integer bounds and re-presolving
Presolve 32586 (-8279) rows, 24796 (-8270) columns and 215181 (-87058) elements
Statistics for presolved model
Original problem has 1413 integers (0 of which binary)
Presolved problem has 1395 integers (0 of which binary)
==== 24486 zero objective 13 different
==== absolute objective values 13 different
==== for integers 1386 zero objective 10 different
1386 variables have objective of -0
1 variables have objective of 969.608
1 variables have obje

In [68]:
ubound = -result_obj['Problem'][0]['Upper bound']
objective = instance.Objective.value
gap = 1 - objective/ubound
print(result_obj['Solver'])
print(f"Gap: {gap}\nUBound: {ubound}\nObjective: {objective}")


- Status: warning
  User time: -1.0
  System time: 591.16
  Wallclock time: 610.38
  Termination condition: intermediateNonInteger
  Termination message: Optimization terminated because a limit was hit, however it had not found an integer solution yet.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 610.5994830131531

Gap: 0.0
UBound: 962282.03
Objective: 962282.03


## Results Analysis

A helper class has been created to help with the analysis of the proposed solution. 

In [43]:
from src.results import ResultsAnalyzer

In [44]:

analyzer = ResultsAnalyzer(instance)
# The first type of plot shows all simulated time series
# All plotting funcitons need to be given the name of the variable/parameter that wants to be analyzed  
analyzer.plot_ts(name='vTotalWealth', time_col='time',col_names=['scenario', 'time'], colors='scenario', inline_plot=True)

In [45]:
# This second plot shows the distribution of any variable/parameter
analyzer.plot_dist(name='vTotalWealth', col_names=['scenario', 'time'], filter={'time':5}, inline_plot=True)


In [52]:
# This third plot shows the time series evolution with a certain confidence interval
# Note how the expected returns are highly influenced by a few outliers with excellent returns 
analyzer.plot_ci(name='vTotalWealth', time_col='time', scenario_col='scenario',filter=None, col_names=['scenario', 'time'], confidence=.50, inline_plot=True)

In [47]:
# Any variable can also be retrieved as a dataframe
df = analyzer.get_df(name='vNonCashAllocations',col_names=['scenario', 'time'])
df[df['value']!=0]

,scenario,time,value
88,^DJI,35,0.009750
89,^DJI,36,0.011892
90,^DJI,37,0.034191
91,^DJI,38,0.037008
92,^DJI,39,0.039864
...,...,...,...
472,^GDAXI,48,0.050103
473,^GDAXI,49,0.050451
474,^GDAXI,50,0.050769
475,^GDAXI,51,0.051221


In [48]:
analyzer.plot_ci(name='pPrices', time_col='time', scenario_col='scenario',filter={'asset':'^XAU'}, col_names=['scenario', 'asset', 'time'], confidence=.95, inline_plot=True)

In [49]:
analyzer.plot_ci(name='vCashAllocations', time_col='time', scenario_col='scenario',filter=None, col_names=['scenario', 'time'], confidence=.9, inline_plot=True)

In [50]:
analyzer.plot_ci(name='pIncome', time_col='time', scenario_col='scenario',filter=None, col_names=['scenario', 'time'], confidence=.95, inline_plot=True)

Here we see for example how the number of simulated scenarios should probably be higher, since income is a normal distribution independent of time the mean and 95%-tiles should be fairly constant. Here however the effect of individual deviations is still recognizable 

In [51]:
analyzer.plot_ci(name='pExpense', time_col='time', scenario_col='scenario',filter=None, col_names=['scenario', 'time'], confidence=.95, inline_plot=True)